## K-Nearest Neighbors (BaseEstimator, ClassifierMixin)

Para entender por qué se obtienen las etiquetas predichas [0, 1] en el ejemplo, repasemos cada paso y veamos cómo se llega a estas predicciones:

Matriz de Distancia:

$$ [[1.         1.         2.23606798]
 [2.23606798 1.         1.        ]] $$

Esta matriz muestra las distancias entre cada muestra de prueba $(X_test)$ y cada muestra de entrenamiento $(X_train)$. Para $X_test[0]$, las distancias a las muestras de entrenamiento son $[1, 1, 2.23606798]$, y para $X_test[1]$, las distancias son $[2.23606798, 1, 1]$.

Índices de las Posiciones Ordenadas:

$$ [[0 1 2]
 [1 2 0]] $$
Aquí, los índices están ordenados por distancia ascendente para cada muestra de prueba. Para $X_test[0]$, el orden de índices de las muestras más cercanas a más lejanas en X_train es $[0, 1, 2]$. Para $X_test[1]$, es $[1, 2, 0]$.

Tomar los Primeros 2 Índices:

$$ [[0 1]
 [1 2]] $$
Dado que n_neighbors es 2, solo consideramos los índices de las dos muestras más cercanas en X_train para cada muestra en $X_test$. Para $X_test[0]$, las muestras más cercanas son las ubicadas en los índices $[0, 1]$ de X_train, y para $X_test[1]$, son $[1, 2]$.

Seleccionar las Etiquetas y_train Más Cercanas:

$$ [[0 1]
 [1 0]] $$
Basándonos en los índices de las muestras más cercanas, seleccionamos sus etiquetas correspondientes. Para $X_test[0]$, las etiquetas de las muestras más cercanas son $[0, 1]$. Para $X_test[1]$, las etiquetas son $[1, 0]$.

Predicciones:

Para $X_test[0]$, las etiquetas de las muestras más cercanas son $[0, 1]$. La etiqueta más común (la moda) entre estas es 0 (ya que no hay una mayoría clara, se elige la primera encontrada, que es 0).
Para $X_test[1]$, las etiquetas de las muestras más cercanas son $[1, 0]$. La etiqueta más común entre estas es 1 (por la misma razón que en el caso anterior).
Por lo tanto, las etiquetas predichas son $[0, 1]$, lo que significa que el modelo predice que la primera muestra de prueba ($X_test[0]$) pertenece a la clase 0, y la segunda muestra de prueba ($X_test[1]$) pertenece a la clase 1.

In [12]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from collections import Counter

# Example training data (features and labels)
X_train = np.array([[1, 2], [2, 3], [3, 4]])
y_train = np.array([0, 1, 0])

# Example test data
X_test = np.array([[2, 2], [3, 3]])
y_test = np.array([0, 0])

# Number of neighbors to consider
n_neighbors = 2

# Compute pairwise distances from test data to training data
dist_matrix = pairwise_distances(X_test, X_train)
print("Distance Matrix:\n", dist_matrix, "\n")

# Get indices of nearest neighbors for each test data point
dist_sort_pos = np.argsort(dist_matrix, axis=1)
print("Distance sorted positions:\n", dist_sort_pos, "\n")

dist_sort_pos_n_neighbors = dist_sort_pos[:, :n_neighbors]
print("Slice the 2 first:\n", dist_sort_pos_n_neighbors, "\n")

# Gather the labels of the nearest neighbors
y_closest = y_train[dist_sort_pos_n_neighbors]
print("Selecting the Y_train closest:\n", y_closest, "\n")

# Predict the label for each test data point
y_pred = [max(Counter(row), key=Counter(row).get) for row in y_closest]

# Output the predictions
print("Predicted labels:", np.array(y_pred))
print("Actual Labels:", y_test)


Distance Matrix:
 [[1.         1.         2.23606798]
 [2.23606798 1.         1.        ]] 

Distance sorted positions:
 [[0 1 2]
 [1 2 0]] 

Slice the 2 first:
 [[0 1]
 [1 2]] 

Selecting the Y_train closest:
 [[0 1]
 [1 0]] 

Predicted labels: [0 1]
Actual Labels: [0 0]


In [13]:
def score(y_true, y_pred):
    """Calculate the accuracy of predictions.

    Parameters
    ----------
    y_true : ndarray, shape (n_samples,)
        True labels.
    y_pred : ndarray, shape (n_samples,)
        Predicted labels.

    Returns
    -------
    score : float
        Accuracy of the predictions.
    """
    correct = (y_true == y_pred).sum()
    return correct / len(y_true)

# Calculate the accuracy
accuracy = score(y_test, np.array(y_pred))
print("Accuracy:", accuracy)

Accuracy: 0.5


## MONTHLY SPLIT

In [28]:
import pytest
import numpy as np
import pandas as pd
from numpy.testing import assert_array_equal

from sklearn.utils.estimator_checks import check_estimator
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier

from sklearn_questions import KNearestNeighbors
#from sklearn_questions import MonthlySplit


date = pd.date_range(start='2020-01-01', end="2023-01-01", freq='D')
n_samples = len(date)
X = pd.DataFrame(range(n_samples), index=date, columns=['val'])
y = pd.DataFrame(
    np.array([i % 2 for i in range(n_samples)]),
    index=date
    )
X = X.rename_axis('Date').reset_index()


In [46]:
import pandas as pd
from sklearn.model_selection import BaseCrossValidator
from pandas.api.types import is_datetime64_any_dtype

class MonthlySplit(BaseCrossValidator):
    def __init__(self, time_col='index'):  # noqa: D107
        self.time_col = time_col

    def get_n_splits(self, X, y=None, groups=None):
        X = X.reset_index()

        if not is_datetime64_any_dtype(X[self.time_col]):
            raise ValueError("Not in a datetimeFormat")

        date = X[self.time_col]
        date_y_m = date.apply(lambda x: str(x.year) + str(x.month))

        return date_y_m.unique().shape[0] - 1

    def split(self, X, y, groups=None):
        X = X.reset_index()
        X = X.sort_values(by=self.time_col)
        n_splits = self.get_n_splits(X, y, groups)
        date = X[self.time_col]
        date_y_m = date.apply(lambda x: str(x.year) + str(x.month)).unique()
        for i in range(n_splits):
            year, month = int(date_y_m[i][0:4]), int(date_y_m[i][4:])
            train_idx = X[
                (date.dt.year == year) & (date.dt.month == month)
                ].index.to_numpy()
            year, month = int(date_y_m[i+1][0:4]), int(date_y_m[i+1][4:])
            test_idx = X[
                (date.dt.year == year) & (date.dt.month == month)
                ].index.to_numpy()

            yield train_idx, test_idx


